In [2]:
import os
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

We first read in our train and test data. We assume that all the data are storedd as csv files in a separate `data` directory.

In [3]:
DIR_TRAIN_BY_STATE = "./data/train_by_state/"

train_data = pd.read_csv('./data/train.csv', engine='python').filter(items=['ID', 'Province_State', 'Date', 'Confirmed', 'Deaths'])
test_data = pd.read_csv('./data/test.csv', engine='python') 

Since we will potentially have a different model for every state, for convenience, we separate the train data into respective states to accelerate the learning steps.

In [4]:
# Get list of state names
states_names = np.unique(np.array([train_data['Province_State']]))
assert(len(states_names) == 50)
print(states_names)

def split_train_data_by_state(train_data):
    for state in states_names:
        state_data = train_data[train_data['Province_State'] == state]
        csv_name = DIR_TRAIN_BY_STATE + state + ".csv"
        state_data.to_csv(csv_name)


['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee' 'Texas' 'Utah'
 'Vermont' 'Virginia' 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming']


But we only want to do this if we haven't done it already.

In [5]:
if not os.path.exists(DIR_TRAIN_BY_STATE):
    os.mkdir(DIR_TRAIN_BY_STATE)
    
if not len(os.listdir(DIR_TRAIN_BY_STATE)):
    split_train_data_by_state(train_data)

To get the best hyperparameters, we generate candidates to do a grid search for the one with best performance.

In [6]:
p = range(1, 5)
d = [1]
q = range(1, 8)
pdq_candidates = list(itertools.product(p, d, q))
seasonal_pdq_candidates = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print(pdq_candidates)
print(seasonal_pdq_candidates)

[(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 1, 5), (1, 1, 6), (1, 1, 7), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 1, 6), (2, 1, 7), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 1, 4), (3, 1, 5), (3, 1, 6), (3, 1, 7), (4, 1, 1), (4, 1, 2), (4, 1, 3), (4, 1, 4), (4, 1, 5), (4, 1, 6), (4, 1, 7)]
[(1, 1, 1, 12), (1, 1, 2, 12), (1, 1, 3, 12), (1, 1, 4, 12), (1, 1, 5, 12), (1, 1, 6, 12), (1, 1, 7, 12), (2, 1, 1, 12), (2, 1, 2, 12), (2, 1, 3, 12), (2, 1, 4, 12), (2, 1, 5, 12), (2, 1, 6, 12), (2, 1, 7, 12), (3, 1, 1, 12), (3, 1, 2, 12), (3, 1, 3, 12), (3, 1, 4, 12), (3, 1, 5, 12), (3, 1, 6, 12), (3, 1, 7, 12), (4, 1, 1, 12), (4, 1, 2, 12), (4, 1, 3, 12), (4, 1, 4, 12), (4, 1, 5, 12), (4, 1, 6, 12), (4, 1, 7, 12)]


In [7]:
def mape(pred, gt):
    ape = np.abs(pred - gt) / np.abs(gt)
    return np.mean(ape) * 100

In [20]:
def train_valid_split(discard_date="04-12-2020", split_date="08-06-2020"):
    valid_data = train_data[(train_data['Date'] >= split_date) & (train_data['Date'] <= "08-31-2020")]
    train_set = train_data[(train_data['Date'] >= discard_date) & (train_data['Date'] < split_date)]
#     print(train_set.head())

    valid_confirmed_dict = {}    
    valid_death_dict = {}
    train_confirmed_dict = {}    
    train_death_dict = {}

    for state in states_names:
        state_valid = valid_data[valid_data["Province_State"] == state]
        state_train = train_set[train_set["Province_State"] == state]

        state_valid_c = np.array(state_valid["Confirmed"].tolist())
        state_valid_d = np.array(state_valid["Deaths"].tolist())
        valid_confirmed_dict[state] = state_valid_c
        valid_death_dict[state] = state_valid_d
        
        state_train_c = np.array(state_train["Confirmed"].tolist())
        state_train_d = np.array(state_train["Deaths"].tolist())
        train_confirmed_dict[state] = state_train_c
        train_death_dict[state] = state_train_d

    return train_confirmed_dict, train_death_dict, valid_confirmed_dict, valid_death_dict


In [22]:
train_confirmed_dict, train_death_dict, valid_confirmed_dict, valid_death_dict = train_valid_split()
train_confirmed_dict["Alabama"].shape

   ID Province_State        Date  Confirmed  Deaths
0   0        Alabama  04-12-2020       3563      93
1   1         Alaska  04-12-2020        272       8
2   2        Arizona  04-12-2020       3542     115
3   3       Arkansas  04-12-2020       1280      27
4   4     California  04-12-2020      22795     640


(117,)

In [13]:
def run_arima():
    predictions = []
    for state in states_names:
        state_data = pd.read_csv(DIR_TRAIN_BY_STATE + state + ".csv")
        
        mape_confirmed = 1e7
        pdq_confirmed = None
        model_confirmed = None

        mape_death = 1e7
        pdq_death = None
        model_death = None

        for pdq in pdq_candidates:
            mod = ARIMA(train_confirmed_dict[state], order=pdq, enforce_stationarity=False)
            f = mod.fit()
            pred_c = f.predict(start=train_confirmed_dict[state].shape[0], end=train_confirmed_dict[state].shape[0] + valid_confirmed_dict[state].shape[0] - 1)
            error = mape(np.array(pred_c.tolist()), valid_confirmed_dict[state])
            if error < mape_confirmed:
                print("Updating param: ", error, pdq)
                mape_confirmed = error
                pdq_confirmed = pdq

        print("Best parameter for ", state, "'s confirmed is PDQ: ", pdq_confirmed)
        model_confirmed = ARIMA(state_data['Confirmed'], order=pdq_confirmed, enforce_stationarity=False)

        for pdq in pdq_candidates:
            mod = ARIMA(train_death_dict[state], order=pdq, enforce_stationarity=False)

            f = mod.fit()
            pred_d = f.predict(start=train_death_dict[state].shape[0], end=train_death_dict[state].shape[0] + valid_death_dict[state].shape[0] - 1)
            error = mape(np.array(pred_d.tolist()), valid_death_dict[state])
            if error < mape_death:
                print("Updating param: ", error, pdq)
                mape_death = error
                pdq_death = pdq
        
        print("Best parameter for ", state, "'s death is PDQ: ", pdq_death)
        model_death = ARIMA(state_data['Deaths'], order=pdq_death, enforce_stationarity=False)

        fit_confirmed = model_confirmed.fit()
        predict_confirmed = fit_confirmed.predict(start=142, end=167)
        fit_death = model_death.fit()

        predict_death = fit_death.predict(start=142, end=167)
        predictions.append((np.around(predict_confirmed, decimals=0), np.around(predict_death, decimals=0)))

    return predictions

In [14]:
def run_seasonal_arima():
    predictions = []
    for state in states_names:
        state_data = pd.read_csv(DIR_TRAIN_BY_STATE + state + ".csv")

        mape_confirmed = 1e7
        pdq_confirmed = None
        seasonal_pdq_confirmed = None
        model_confirmed = None

        mape_death = 1e7
        pdq_death = None
        seasonal_pdq_death = None
        model_death = None

        for pdq in pdq_candidates:
            for seasonal_pdq in seasonal_pdq_candidates:
                try:
                    mod = SARIMAX(train_confirmed_dict[state], order=pdq, seasonal_order=seasonal_pdq,
                                  enforce_stationarity=False, enforce_invertibility=False)

                    f = mod.fit(disp=False, method='powell')
                    pred_c = f.predict(start=train_confirmed_dict[state].shape[0], end=train_confirmed_dict[state].shape[0] + valid_confirmed_dict[state].shape[0] - 1)
                    error = mape(np.array(pred_c.tolist()), valid_confirmed_dict[state])
                    if error < mape_confirmed:
                        print("Updating param: ", error, pdq_confirmed, seasonal_pdq_confirmed)
                        mape_confirmed = error
                        pdq_confirmed = pdq
                        seasonal_pdq_confirmed = seasonal_pdq
                except:
                    continue
       
        print("Best parameter for ", state, "'s confirmed is PDQ: ", pdq_confirmed, " and Seasonal PDQ: ", seasonal_pdq_confirmed)
        model_confirmed = SARIMAX(state_data['Confirmed'], order=pdq_confirmed, seasonal_order=seasonal_pdq_confirmed,
                                  enforce_stationarity=False, enforce_invertibility=False)

        for pdq in pdq_candidates:
                    for seasonal_pdq in seasonal_pdq_candidates:
                        try:
                            mod = SARIMAX(train_death_dict[state], order=pdq, seasonal_order=seasonal_pdq,
                                          enforce_stationarity=False, enforce_invertibility=False)
                            f = mod.fit(disp=False, method='powell')
                            pred_d = f.predict(start=train_death_dict[state].shape[0], end=train_death_dict[state].shape[0] + valid_death_dict[state].shape[0] - 1)
                            error = mape(np.array(pred_d.tolist()), valid_death_dict[state])
                            if error < mape_death:
                                print("Updating param: ", error, pdq, seasonal_pdq)
                                mape_death = error
                                pdq_death = pdq
                                seasonal_pdq_death = seasonal_pdq
                        except:
                            continue
        
        print("Best parameter for ", state, "'s deaths is PDQ: ", pdq_death, " and Seasonal PDQ: ", seasonal_pdq_death)
        model_death = SARIMAX(state_data['Deaths'],order=pdq_death, seasonal_order=seasonal_pdq_death,
                                  enforce_stationarity=False, enforce_invertibility=False)

        fit_c = model_confirmed.fit(disp=False, method='powell')
        y_pred_confirmed = fit_c.predict(start=142, end=167)
        fit_d = model_death.fit(disp=False, method='powell')

        y_pred_deaths = fit_d.predict(start=142, end=167)
        predictions.append((np.around(y_pred_confirmed, decimals=0), np.around(y_pred_deaths, decimals=0)))

        return predictions

In [ ]:
predictions = run_arima()
# predictions = run_seasonal_arima()

predictions

Updating param:  3.2340492434643973 None None
Updating param:  2.866304217248371 (1, 1, 1) (1, 1, 1, 12)


C:\Users\tonyl\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Updating param:  2.632335155589353 (1, 1, 1) (1, 1, 2, 12)


In [214]:
test_submission = test_data.sort_values(["Province_State", "Date"])
confirmed = []
deaths = []
for i in range(50):
    confirmed.append(predictions[i][0].astype(int).tolist())
    deaths.append(predictions[i][1].astype(int).tolist())

confirmed = list(itertools.chain.from_iterable(confirmed))
deaths = list(itertools.chain.from_iterable(deaths))

test_submission.loc[:, "Confirmed"] = confirmed
test_submission.loc[:, "Deaths"] = deaths

test_submission = test_submission.sort_index().filter(items=['ForecastID', 'Confirmed', 'Deaths'])
test_submission.to_csv("Team15_arima.csv", index=False)

Seasonal Arima was meant to apply seasonal effect to better capture the trend in COVID 19 growth. However, in practice add seasonal parameters massively increased running time and did not produce better results. The best score we were able to achieve is about 2.36